In [1]:
%matplotlib inline

In [96]:
import string

import numpy as np
import pandas as pd
import nltk

from functools import reduce
from collections import Counter

Initial file read and check

In [163]:
df_mont = pd.read_csv('../data/montgomery_dh_1.csv')
df_wiki = pd.read_csv('../data/wikipedia_celebrity_deaths_4.csv')

In [154]:
print df_mont.shape
# dh_csv.head()

(158512, 4)


In [155]:
print df_wiki.shape
# wiki_csv.head()

(21458, 9)


In [156]:
# df_wiki.birth_year.value_counts()
# df_wiki[df_wiki.birth_year < 1900]
# df_wiki.death_year.value_counts()

In [157]:
# df_wiki.nationality.value_counts()

In [159]:
df_wiki.head()
# df_wiki[df_wiki.nationality == 'Libyan']

,age,birth_year,cause_of_death,death_month,death_year,famous_for,name,nationality,fame_score
0,85,1921,natural causes,January,2006,businessman chairman of IBM (1973���1981),Frank Cary,American,1
1,49,1957,murdered,January,2006,musician (House of Freaks Gutterball),Bryan Harvey,American,2
2,64,1942,Alzheimer's disease,January,2006,baseball player (Oakland Athletics),Paul Lindblad,American,1
3,86,1920,Alzheimer's disease,January,2006,politician Representative from Oregon (1957��...,Charles O. Porter,American,2
4,82,1924,cancer,January,2006,nightclub owner (Tropicana Club),Ofelia Fox,Cuban,NaN


In [165]:
df_wiki['nationality'] = df_wiki.nationality.map(lambda text: text.replace("-born", ""))

In [166]:
df_wiki.nationality.value_counts()

American             8068
British              1506
Canadian              916
English               628
Australian            619
Italian               553
German                537
French                448
Indian                402
Russian               376
Norwegian             292
Dutch                 280
Brazilian             273
Japanese              267
Irish                 266
Spanish               249
Scottish              237
Polish                230
Swedish               207
Mexican               204
New                   183
Hungarian             168
Argentine             160
South                 136
Belgian               128
Austrian              122
Romanian              119
Filipino              118
Czech                 113
Chinese               105
                     ... 
built                   1
Violinist               1
Greek-Cypriot           1
Sandinista              1
Chief                   1
Environmental           1
Guernsey                1
English-Amer

In [125]:
keyword = "parliament"
df_wiki[df_wiki.famous_for.map(lambda text: keyword in str(text))]

,age,birth_year,cause_of_death,death_month,death_year,famous_for,name,nationality,fame_score,fame_text
882,62,1945,cancer,August,2007,member of parliament,Bahaedin Adab,Irania,2,member of parliament
6521,59,1952,NaN,April,2011,parliamentary leader of One Nation (2001���2004),Bill Flynn,Australia,2,parliamentary leader of One Nation 2001���2004
8287,88,1924,NaN,February,2012,member of parliament (1973���1977),Reidar T. Larsen,Norwegia,3,member of parliament 1973���1977
13458,79,1935,stroke,April,2014,politician Senator for Tasmania (1975���2005)...,Brian Harradine,Australia,14,politician Senator for Tasmania 1975���2005 l...


In [128]:
keyword = "born"
df_wiki[df_wiki.nationality.map(lambda text: keyword in str(text))]

,age,birth_year,cause_of_death,death_month,death_year,famous_for,name,nationality,fame_score,fame_text


Don't do this:

all_text = []
word_counter = {}
df_wiki['fame_text'] = df_wiki.famous_for.map(
    lambda text: "".join([w.translate(None, string.punctuation) for w in str(text).split(" ")])

for idx, row in df_wiki.iterrows():
    all_text += extend(row['fame_text'])
    
all_text = [word for word in all_text if word != '']

This is more efficient:

In [129]:
all_text_string = ""
word_counter = {}
df_wiki['fame_text'] = df_wiki.famous_for.map(
    lambda text: " ".join([w.translate(None, string.punctuation) for w in str(text).split(" ")]))

for idx, row in df_wiki.iterrows():
    all_text_string += row['fame_text']

In [12]:
def get_word_counts(text_block):
    word_list = nltk.word_tokenize(text_block)
    word_counts = {}
    for word in word_list:
        if word in word_counts.keys():
            word_counts[word] += 1
        else:
            word_counts[word] = 1
    print "Number of words:", len(word_counts)
    return word_counts

In [13]:
%%time
word_counter = get_word_counts(all_text_string)

Number of words: 16526
CPU times: user 49 s, sys: 492 ms, total: 49.5 s
Wall time: 51.2 s


In [14]:
word_count_table = pd.DataFrame.from_dict(word_counter, orient='index')
word_count_table.reset_index(inplace=True)
word_count_table.columns = ['word', 'count']

In [15]:
word_count_table['part_of_speech'] = word_count_table.word.map(
    lambda word: nltk.pos_tag([word])[0][1])

In [16]:
noun_type_list = ["NN", "NP"]
word_count_table[word_count_table.part_of_speech.map(
        lambda e: e in noun_type_list)].sort('count', ascending=False).head(300)

,word,count,part_of_speech
702,player,2649,NN
10984,politician,1788,NN
965,actor,1551,NN
7978,footballer,1243,NN
13385,football,1221,NN
16217,member,1216,NN
15032,Olympic,1147,NN
14201,actress,881,NN
13530,baseball,776,NN
5067,Bishop,770,NN


Notes:
- "MP" (more frequently used) stands for "member of parlianemt" (less frequently used)

### Nationality mapping

Using list compiled from [here](https://github.com/Dinu/country-nationality-list/blob/master/countries.csv) which in turn grabs from [Wikipedia's page](https://en.wikipedia.org/wiki/List_of_adjectival_and_demonymic_forms_for_countries_and_nations) for adjectival and demonymic forms for countries and nations.

In [85]:
natl_mapping = pd.read_csv('../data/countries.csv')
print natl_mapping.shape
natl_mapping.head()

(249, 5)


,num_code,alpha_2_code,alpha_3_code,en_short_name,nationality
0,4,AF,AFG,Afghanistan,Afghan
1,248,AX,ALA,Åland Islands,Åland Island
2,8,AL,ALB,Albania,Albanian
3,12,DZ,DZA,Algeria,Algerian
4,16,AS,ASM,American Samoa,American Samoan


In [86]:
text_filter = (lambda text: " or " in text)
idx_with_text = natl_mapping.nationality.map(text_filter)
natl_mapping[idx_with_text]['nationality_text_list'] = \
   natl_mapping[idx_with_text].nationality.map(lambda text: text.replace(" or ", ", "))

/Users/George/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


List of nationalities:

In [87]:
"""
Run once
"""
natl_mapping.nationality = \
    natl_mapping.nationality.map(lambda text: text.replace(" or ", ", ").split(", "))

In [104]:
nationality_list = [label 
                    for label_list in natl_mapping.nationality.values 
                    for label in label_list]
Counter(nationality_list)
# Chinese, American, Dominican, Congolese, Channel Island

Counter({'Chinese': 3, 'American': 2, 'Dominican': 2, 'Congolese': 2, 'Channel Island': 2, 'Cook Island': 1, 'South Sudanese': 1, 'Zimbabwean': 1, 'Salvadoran': 1, 'Panamanian': 1, 'Serbian': 1, 'Chadian': 1, 'Grenadian': 1, 'Palauan': 1, 'Kittitian': 1, 'Malinese': 1, 'Saint Helenian': 1, 'Saint-Martinoise': 1, 'Central African': 1, 'Kyrgyz': 1, 'Aruban': 1, 'Kenyan': 1, 'New Zealand': 1, 'Argentine': 1, 'Belgian': 1, 'Bulgarian': 1, 'Azeri': 1, 'UK': 1, 'Timorese': 1, 'Polish': 1, 'Herzegovinian': 1, 'Seychellois': 1, 'French': 1, 'Peruvian': 1, 'Fijian': 1, 'Liberian': 1, 'Armenian': 1, 'Guyanese': 1, 'Monacan': 1, 'Qatari': 1, 'Haitian': 1, 'Vincentian': 1, 'Samoan': 1, 'Mauritanian': 1, 'Finnish': 1, 'Canadian': 1, 'Manx': 1, 'Albanian': 1, 'Omani': 1, 'Comoran': 1, 'Romanian': 1, 'Maltese': 1, 'Andorran': 1, 'Bahraini': 1, 'Martinican': 1, 'Vatican': 1, 'Wallis and Futuna': 1, 'Kirgiz': 1, 'NZ': 1, 'Emirian': 1, 'Italian': 1, 'Malian': 1, 'Ivorian': 1, 'Norfolk Island': 1, 'Saudi

In [89]:
natl_mapping.head()

,num_code,alpha_2_code,alpha_3_code,en_short_name,nationality
0,4,AF,AFG,Afghanistan,[Afghan]
1,248,AX,ALA,Åland Islands,[Åland Island]
2,8,AL,ALB,Albania,[Albanian]
3,12,DZ,DZA,Algeria,[Algerian]
4,16,AS,ASM,American Samoa,[American Samoan]


In [110]:
nationality_map_dict = {}
for idx, row, in natl_mapping.iterrows():
    for label in row['nationality']:
        row_info_entry = [row['num_code'], row['alpha_2_code'],
                          row['alpha_3_code'], row['en_short_name']]
        if label in nationality_map_dict.keys():
            nationality_map_dict[label].append(row_info_entry)
        else:
            nationality_map_dict[label] = [row_info_entry]

In [135]:
nationality_map_dict['British']

[[826, 'GB', 'GBR', 'United Kingdom of Great Britain and Northern Ireland']]

In [122]:
natl_mapping[natl_mapping.nationality == 'American']

,num_code,alpha_2_code,alpha_3_code,en_short_name,nationality
